# 2 基于jsonpath-ng的进阶JSON数据处理方法

## 2.1 JSON数据的增删改

In [1]:
import json
from jsonpath_ng import parse

# 读入示例json数据
with open('json示例.json', encoding='utf-8') as j:
    demo_json = json.loads(j.read())
    
# 构造指定JSONPath模式对应的解析器
parser = parse('$..paths..steps[*].duration')

# 利用解析器的find方法找到目标数据中所有满足条件的节点
matches = parser.find(demo_json)

# 利用value属性取得对应匹配结果的值
matches[0].value

'59'

### 2.1.1 对JSON数据进行增操作

In [2]:
# 构造示例数据
demo_json = {
    'level1': [
        {
            'level2': {}
        },
        {
            'level2': {
                'level3': 12
            }
        }
    ]
}

# 构造规则解释器，所有除去最后一层节点规则外可以匹配到的节点
# 都属于合法匹配结果，会在匹配结果列表中出现
parser = parse('level1[*].level2.level3')

matches = parser.find_or_create(demo_json)

demo_json

{'level1': [{'level2': {'level3': {}}}, {'level2': {'level3': 12}}]}

In [3]:
for match in matches:
    if match.value == {}:
        # 更新原始输入的JSON数据
        match.full_path.update_or_create(demo_json, 999)

demo_json

{'level1': [{'level2': {'level3': 999}}, {'level2': {'level3': 12}}]}

### 2.1.2  对JSON数据进行删操作

In [4]:
parser = parse('level1[*].level2.level3')

# 过滤 level1[*].level2.level3 规则下值为 999 的节点
parser.filter(lambda x: x == 999, demo_json)
demo_json

{'level1': [{'level2': {}}, {'level2': {'level3': 12}}]}

### 2.1.3  对JSON数据进行改操作

In [5]:
parser = parse('level1[*].level2.level3')

# 以 只更新全路径匹配结果 的策略修改JSON数据
parser.update(demo_json, 100)

{'level1': [{'level2': {}}, {'level2': {'level3': 100}}]}

In [6]:
# 使用 有则更新，无则新建 的策略修改JSON数据
parser.update_or_create(demo_json, 200)

{'level1': [{'level2': {'level3': 200}}, {'level2': {'level3': 200}}]}